In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import matplotlib.pyplot as plt
from transformers import *
%matplotlib inline

/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
# Train Dataset

# id: rowID
# qid{1, 2}: unique ID of each question in the pair
# question{1, 2}: textual contents of the questions.
# is_duplicate: target, whether the two questions are duplicates of each other.

#df_train = pd.read_csv("../input/quora-question-pairs/train.csv.zip").fillna("")
df_train = pd.read_csv("data/train.csv").fillna("")
df_train.head()

df_test = pd.read_csv("data/test.csv").fillna("")

/home/danasour/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tr_qids = pd.Series(df_train['qid1'].tolist() + df_train['qid2'].tolist())
qs_tr = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist()).astype(str)
qs_te = pd.Series(df_test['question1'].tolist() + df_test['question2'].tolist()).astype(str)

MODEL_CLASSES = {
    'bert': (           BertForSequenceClassification,          BertTokenizer,          'bert-base-uncased')
}
model_class, tokenizer, pretrained_model = MODEL_CLASSES['bert']

tokenizer = tokenizer.from_pretrained(pretrained_model)

unique_dic = tr_qids.value_counts()
len(unique_dic)

537933

In [4]:
def clusters (llista):
    node_to_class = {}
    class_to_nodes = {}
    for f,s,t in llista:
        if f not in node_to_class:
            node_to_class[f] = f
            class_to_nodes[f] = [f]

        if s not in node_to_class:
            node_to_class[s] = s
            class_to_nodes[s] = [s]
        
        if t and node_to_class[f] != node_to_class[s]: # class join
            class_m = min(node_to_class[f], node_to_class[s])
            class_M = max(node_to_class[f], node_to_class[s])
            
            class_to_nodes[class_m] += class_to_nodes[class_M]
            for node in class_to_nodes[class_M]:
                node_to_class[node] = class_m
            del class_to_nodes[class_M]

    return (class_to_nodes, node_to_class)

(class_to_nodes, node_to_class) = clusters(np.array(df_train[["qid1", "qid2","is_duplicate"]]))
print("There are", len(class_to_nodes), "different question classes")

There are 448743 different question classes


In [5]:
data = np.array(df_train)
sentences = [None]*len(unique_dic)

for row in data:
    for i in range(1,3):
        id = row[i]
        if not sentences[id-1]:
            question = row[i+2]
            sentences[id-1] = [id-1,question]

In [82]:
N_sizes = 10
N_classes = 5
count = [N_classes]*N_sizes

embe
first = True

for key, value in class_to_nodes.items():
    size = len(value)
    if size >= 2 and size < N_sizes+2 and count[size-2] > 0:
        count[size-2] -= 1
        values = [None]*len(value)
        for val in value:
            question = sentences[val-1]
            embedding = tokenizer.encode(question[1]) 
            if len(embedding) > 34:
                embedding = embedding[0:34]
            else:
                embedding.extend([0] * (34 - len(embedding)))
            
            emb = pd.DataFrame([embedding])
            q = pd.DataFrame([val, key, question[1]], index=["id", "class","question"]).transpose()
            if first:
                res = pd.concat([q, emb], axis=1)
                first = False
            else:
                res = res.append(pd.concat([q, emb], axis=1))

In [83]:
res.shape

(325, 37)

In [97]:
embeddings = res.iloc[:, 3:(3+34)]

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,101,28625,6483,1024,1045,2572,1037,6178,7277,9691,...,2033,1029,102,0,0,0,0,0,0,0
0,101,1045,1005,1049,1037,6420,6178,7277,9691,1006,...,2023,2360,2055,2033,1029,102,0,0,0,0
0,101,2129,2064,1045,2022,1037,2204,21334,1029,102,...,0,0,0,0,0,0,0,0,0,0
0,101,2054,2323,1045,2079,2000,2022,1037,2307,21334,...,0,0,0,0,0,0,0,0,0,0
0,101,2129,2079,1045,3191,1998,2424,2026,7858,7928,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,101,2339,2079,2796,7486,5223,16913,2072,1029,102,...,0,0,0,0,0,0,0,0,0,0
0,101,2339,2079,2796,7486,5223,1052,1012,1049,1012,...,0,0,0,0,0,0,0,0,0,0
0,101,2003,2009,2995,2035,7486,5223,16913,2072,1029,...,0,0,0,0,0,0,0,0,0,0
0,101,2339,2515,2116,7486,5223,16913,2072,1029,102,...,0,0,0,0,0,0,0,0,0,0


In [103]:
from sklearn.manifold import TSNE
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import sklearn.cluster as sk
from mpl_toolkits.mplot3d import Axes3D 

num_clusters = 5

tsne = TSNE(n_components=2,verbose=1, perplexity=40, n_iter=300, random_state=510)
node_embeddings = tsne.fit_transform(np.array(embeddings))

kmeans = sk.KMeans(n_clusters=num_clusters, max_iter =30)
kmeans.fit(node_embeddings)

labels = kmeans.predict(node_embeddings)
centers = kmeans.cluster_centers_

res['x'] = node_embeddings[:,0]
res['y'] = node_embeddings[:,1]

fig_2d = px.scatter(res, x="x", y="y", title= "Question embeddings", hover_data=["id","class", "question"], color= labels)
fig_2d.show()

plotly.offline.plot(fig_2d, filename='2d_tsne.html')

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 325 samples in 0.001s...
[t-SNE] Computed neighbors for 325 samples in 0.020s...
[t-SNE] Computed conditional probabilities for sample 325 / 325
[t-SNE] Mean sigma: 1822.862310
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.875732
[t-SNE] KL divergence after 300 iterations: 0.966853


'2d_tsne.html'

In [106]:
res

,id,class,question,0,1,2,3,4,5,6,...,26,27,28,29,30,31,32,33,x,y
0,11,11,Astrology: I am a Capricorn Sun Cap moon and c...,101,28625,6483,1024,1045,2572,1037,...,102,0,0,0,0,0,0,0,-2.937288,-4.936165
0,12,11,"I'm a triple Capricorn (Sun, Moon and ascendan...",101,1045,1005,1049,1037,6420,6178,...,2055,2033,1029,102,0,0,0,0,10.916789,-6.665578
0,15,15,How can I be a good geologist?,101,2129,2064,1045,2022,1037,2204,...,0,0,0,0,0,0,0,0,-9.600402,-6.190775
0,16,15,What should I do to be a great geologist?,101,2054,2323,1045,2079,2000,2022,...,0,0,0,0,0,0,0,0,8.356997,-0.253423
0,23,23,How do I read and find my YouTube comments?,101,2129,2079,1045,3191,1998,2424,...,0,0,0,0,0,0,0,0,3.187280,-0.857256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,135605,1829,Why do Indian Muslims hate Modi?,101,2339,2079,2796,7486,5223,16913,...,0,0,0,0,0,0,0,0,12.870167,2.573771
0,186640,1829,Why do Indian Muslims hate P.M. Narendra Modi?,101,2339,2079,2796,7486,5223,1052,...,0,0,0,0,0,0,0,0,-5.108050,8.849332
0,68879,1829,Is it true all Muslims hate Modi?,101,2003,2009,2995,2035,7486,5223,...,0,0,0,0,0,0,0,0,-8.912108,-6.076416
0,68880,1829,Why does many Muslims hate Modi?,101,2339,2515,2116,7486,5223,16913,...,0,0,0,0,0,0,0,0,12.874694,2.594684
